In [ ]:
! pip install transformers peft datasets

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from datasets import Dataset
dataset = Dataset.from_json('/home/ubuntu/remove_link_500_500.jsonl')


/home/ubuntu/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "kakaocorp/kanana-nano-2.1b-instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",  # 추가
    low_cpu_mem_usage=True  # 추가
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
def formatting_prompts_func(examples):
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    # 💡 'instruction' 대신 데이터셋 피처인 'introduction'을 사용합니다.
    introductions = examples["introduction"]
    inputs = examples["input"]
    outputs = examples["output"]

    # tokenizer는 해당 셀이 실행되기 전에 정의되어 있어야 합니다.
    EOS_TOKEN = tokenizer.eos_token 

    texts = []
    # 💡 'instruction' 대신 'introduction'을 사용하여 반복문을 실행합니다.
    for introduction, input, output in zip(introductions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        # 💡 instruction 대신 introduction 변수를 format에 전달합니다.
        text = alpaca_prompt.format(introduction, input, output) + EOS_TOKEN
        texts.append(text)

    # 이 함수는 새로운 'text' 열을 데이터셋에 추가합니다.
    return {"text": texts}
dataset = dataset.map(formatting_prompts_func, batched=True)
dataset

Dataset({
    features: ['introduction', 'input', 'output', 'text'],
    num_rows: 1002
})

In [4]:
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"], 
        padding=True, 
        truncation=True,  # 추가
        max_length=4096,  # 추가 (필요에 따라 조정)
        return_tensors="pt"
    )
    tokens["labels"] = tokens["input_ids"]
    return tokens

dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map: 100%|███████████████████████████████████████████████████████████████| 1002/1002 [00:03<00:00, 286.86 examples/s]


In [26]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 모델 준비
base_model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=64,
    lora_dropout=0.1,
    target_modules=[
        "q_proj", 
        "k_proj", 
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj" 
    ],
    bias="none",
)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters() 

trainable params: 23,003,136 || all params: 2,109,982,464 || trainable%: 1.0902


In [27]:
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4, 
        bf16=True,
        logging_steps=1,
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=1234,
        output_dir="outputs",
        report_to="none",
        ddp_find_unused_parameters=False
    )
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [28]:
trainer_stats = trainer.train()

Step,Training Loss
1,7.660400
2,6.942900
3,4.387000
4,2.665400
5,1.118400
6,0.711600
7,1.099500
8,1.019400
9,0.840100
10,1.276500


In [30]:
FINAL_ADAPTER_PATH = "./final_lora_adapter"

model.save_pretrained(FINAL_ADAPTER_PATH) 

print(f" 최종 LoRA 어댑터 가중치가 {FINAL_ADAPTER_PATH}에 저장되었습니다.")

 최종 LoRA 어댑터 가중치가 ./final_lora_adapter에 저장되었습니다.


In [33]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL_ID = "kakaocorp/kanana-nano-2.1b-instruct"
MERGED_MODEL_DIR = "./merged_full_model_for_gguf" # 병합된 모델을 저장할 디렉토리 이름
# ----------------------------------------------------------------------

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# 2. 기본 모델에 LoRA 어댑터를 연결합니다.
model = PeftModel.from_pretrained(base_model, FINAL_ADAPTER_PATH)

# 3. LoRA 가중치를 기본 모델에 영구적으로 병합합니다.
model = model.merge_and_unload() 

# 4. 병합된 Full 모델을 Hugging Face 형식으로 저장합니다.
model.save_pretrained(MERGED_MODEL_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_MODEL_DIR)

print(f" 모델 병합 완료. 최종 파일은 {MERGED_MODEL_DIR}에 저장되었습니다.")

✅ 모델 병합 완료. 최종 파일은 ./merged_full_model_for_gguf에 저장되었습니다.


In [46]:
# llama.cpp 저장소 클론 및 이동
!cd llama && pip install -r ./requirements/requirements-convert_hf_to_gguf.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 14.1 MB/s  0:00:01 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 69.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 MB 123.0 MB/s  0:00:010:00:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0━━━━━ 0/6 [sympy]
  Attempting uninstall: numpy━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [sentencepiece]
    Found existing installation: numpy 2.3.4━━━━━━━━━━━━━━━━━━ 1/6 [sentencepiece]
    Uninstalling numpy-2.3.4:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [sentencepiece]
      Successfully uninstalled numpy-2.3.4━━━

In [53]:
MERGED_MODEL_DIR="./merged_full_model_for_gguf" 
OUTPUT_GGUF_FILE="./my_lora_finetuned_model.gguf"

# 1. Hugging Face 모델을 GGUF 형식으로 변환합니다. (FP16 정밀도로 저장)
# 이 과정은 양자화 전 준비 단계입니다.
!python ./llama.cpp/convert_hf_to_gguf.py $MERGED_MODEL_DIR --outtype f16

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:hf-to-gguf:Loading model: merged_full_model_for_gguf
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {1792, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {1792}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {8064, 1792}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {1792, 8064}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {1792, 8064}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {1792}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {1792, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {3072, 1792}
INFO:hf-to-gguf:blk.0

In [62]:
!cd llama.cpp && mkdir build && cd build && cmake .. && make -j4

/home/ubuntu/llama.cpp


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


-- The C compiler identification is GNU 13.3.0
-- The CXX compiler identification is GNU 13.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.43.0") 
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_

In [71]:
!./llama.cpp/build/bin/llama-quantize ./merged_full_model_for_gguf/Merged_Full_Model_For_Gguf-2.1B-F16.gguf ./model-Q5_K_M.gguf Q5_K_M

main: build = 6912 (961660b8c)
main: built with cc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0 for x86_64-linux-gnu
main: quantizing './merged_full_model_for_gguf/Merged_Full_Model_For_Gguf-2.1B-F16.gguf' to './model-Q5_K_M.gguf' as Q5_K_M
llama_model_loader: loaded meta data with 29 key-value pairs and 290 tensors from ./merged_full_model_for_gguf/Merged_Full_Model_For_Gguf-2.1B-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Merged_Full_Model_For_Gguf
llama_model_loader: - kv   3:                         general.size_label str              = 2.1B
llama_model_loader: - kv   4:                          llama.block_count u32 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[   1/ 290]                   output_norm.weight - [ 1792,     1,     1,     1], type =    f32, size =    0.007 MiB
[   2/ 290]                    token_embd.weight - [ 1792, 128256,     1,     1], type =    f16, converting to q6_K .. size =   438.38 MiB ->   179.80 MiB
[   3/ 290]                  blk.0.attn_k.weight - [ 1792,  1024,     1,     1], type =    f16, converting to q5_K .. size =     3.50 MiB ->     1.20 MiB
[   4/ 290]               blk.0.attn_norm.weight - [ 1792,     1,     1,     1], type =    f32, size =    0.007 MiB
[   5/ 290]             blk.0.attn_output.weight - [ 3072,  1792,     1,     1], type =    f16, converting to q5_K .. size =    10.50 MiB ->     3.61 MiB
[   6/ 290]                  blk.0.attn_q.weight - [ 1792,  3072,     1,     1], type =    f16, converting to q5_K .. size =    10.50 MiB ->     3.61 MiB
[   7/ 290]                  blk.0.attn_v.weight - [ 1792,  1024,     1,     1], type =    f16, converting to q6_K .. size =     3.50 MiB ->     1.44 M